In [ ]:
import yfinance as yf
import pandas as pd


def download_stock_data(ticker, start, end):
    stock_data = yf.download(ticker, start=start, end=end)
    return stock_data


stock_data = download_stock_data('AAPL', '2020-01-01', '2023-01-01')
stock_data.head()



[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,72.876114,135480400
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.167595,146322800
2020-01-06,73.447502,74.989998,73.187500,74.949997,72.742653,118387200
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.400551,108872000
2020-01-08,74.290001,76.110001,74.290001,75.797501,73.565193,132079200


In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler


def preprocess_data(data):

    close_prices = data['Close'].values.reshape(-1, 1)

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(close_prices)

    return scaled_data, scaler


scaled_data, scaler = preprocess_data(stock_data)


In [ ]:
def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)


training_data_len = int(np.ceil(len(scaled_data) * 0.8))
train_data = scaled_data[:training_data_len, :]
X_train, y_train = create_dataset(train_data, 60)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM


def build_model():
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dense(units=25))
    model.add(Dense(units=1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


model = build_model()
model.fit(X_train, y_train, batch_size=64, epochs=10)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - loss: 0.1976
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 0.0211
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 0.0128
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.0066
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 0.0038
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 0.0028
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - loss: 0.0026
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - loss: 0.0022
Epoch 9/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - loss: 0.0020
Epoch 10/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - loss: 0.0020


In [ ]:

test_data = scaled_data[training_data_len - 60:, :]
X_test, y_test = create_dataset(test_data, 60)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)


rmse = np.sqrt(np.mean(((predictions - y_test.reshape(-1, 1)) ** 2)))
print(f"Test RMSE: {rmse}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 205ms/step
Test RMSE: 149.67984731921175


In [ ]:
import time


def get_real_time_data(ticker):
    data = yf.download(ticker, period='1d', interval='1m')
    return data['Close'].iloc[-1]


def predict_real_time_price(model, scaler, recent_data):
    scaled_data = scaler.transform(np.array([recent_data]).reshape(-1, 1))
    scaled_data = np.reshape(scaled_data, (1, scaled_data.shape[0], 1))
    prediction = model.predict(scaled_data)
    return scaler.inverse_transform(prediction)[0][0]


real_time_price = get_real_time_data('AAPL')
prediction = predict_real_time_price(model, scaler, real_time_price)
print(f"Current Stock Price: {real_time_price}, Predicted Next Price: {prediction}")


[*********************100%***********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 462ms/step
Current Stock Price: 228.52999877929688, Predicted Next Price: 69.05987548828125
